# Script para planilha de lista eleitoral

In [1]:
import pypdf
import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns   

from pathlib import Path
from functools import partial
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from utils_ccd import get_connection, extract_text_from_pdf, get_info_file_path
from pathlib import Path
conn = get_connection()

In [54]:
df_lista = pd.read_excel("docs/lista_eleitoral_250527.xlsx")
processos = ','.join(map(lambda x: f"'{x}'", df_lista['NUM_PROCESSO']))

sql_processo = f''' 
SELECT *
FROM processo.dbo.Processos p
WHERE CONCAT(p.numero_processo, '/', p.ano_processo) IN ({processos});
'''
df_processo = pd.read_sql(sql_processo, conn)

sql_all_informacoes_processos = f'''
    SELECT concat(rtrim(inf.setor),'_',inf.numero_processo ,'_',inf.ano_processo,'_',RIGHT(concat('0000',inf.ordem),4),'.pdf') as arquivo,
    ppe.SequencialProcessoEvento as evento,
    CONCAT(inf.numero_processo,'/', inf.ano_processo) as processo,
    inf.*
    FROM processo.dbo.vw_ata_informacao inf INNER JOIN processo.dbo.Pro_ProcessoEvento ppe 
        ON inf.idinformacao = ppe.idinformacao
    WHERE concat(inf.numero_processo, '/', inf.ano_processo) in ({processos})
'''
all_informacoes_processos = pd.read_sql(sql_all_informacoes_processos, conn)

def filter_infos(df):
    df = df.copy()
    df = df[~df['resumo'].str.contains('capa', case=False, na=False)]
    df = df[~df['resumo'].str.contains('digitalizado', case=False, na=False)]
    return df


In [55]:
df_lista.columns = [
    'nome_orgao',
    'esfera',
    'processo',
    'data_processo',
    'nome_requerido',
    'cpf',
    'rg',
    'titulo_eleitoral',
    'cargo',
    'registro_profissional',
    'data_nascimento',
    'nome_mae',
    'endereco',
    'bairro',
    'municipio',
    'uf',
    'cep',
    'data_julgamento',
    'tipo_julgamento',
    'data_condenacao_inicio',
    'data_condenacao_fim',
    'exclusao_suspensa',
    'data_suspensao',
    'assunto_resumo_deliberacao',
    'detalhe_processo',
    'partido',
    'uf_mandato',
    'perda_mandato_renuncia',
    'data_inicio_perda_suspensao',
    'data_fim_perda_suspensao',
    'observacoes',
    'parecer_previo',
    'julgamento_tc',
    'dano',
    'tipo_processo',
    'orgao',
    'campo_livre_6',
    'campo_livre_7'
]


In [56]:
all_informacoes_processos = filter_infos(all_informacoes_processos)

In [57]:
all_informacoes_processos['caminho_arquivo'] = all_informacoes_processos.apply(get_info_file_path, axis=1)

In [58]:
all_informacoes_processos['texto'] = all_informacoes_processos['caminho_arquivo'].apply(extract_text_from_pdf)

Multiple definitions in dictionary at byte 0xfe095 for key /Im16
Multiple definitions in dictionary at byte 0xfe0e3 for key /Im15
Multiple definitions in dictionary at byte 0xfe1a6 for key /Im31
Multiple definitions in dictionary at byte 0xfe235 for key /Im49
Multiple definitions in dictionary at byte 0xfe498 for key /Im93
Multiple definitions in dictionary at byte 0xfe4a5 for key /Im94
Multiple definitions in dictionary at byte 0xfe4b2 for key /Im95
Multiple definitions in dictionary at byte 0xfe520 for key /Im23
Multiple definitions in dictionary at byte 0xfe84a for key /Im106
Multiple definitions in dictionary at byte 0xfe8b3 for key /Im155
Multiple definitions in dictionary at byte 0xfe8c2 for key /Im106
Multiple definitions in dictionary at byte 0xfe8d1 for key /Im156
Multiple definitions in dictionary at byte 0xfe8e0 for key /Im157
Multiple definitions in dictionary at byte 0xfe8ef for key /Im158
Multiple definitions in dictionary at byte 0xfe8fe for key /Im159
Multiple definitio

Error reading \\10.24.0.6\tce$\Informacoes_PDF\GCADE\GCADE_700896_2012_0017.pdf: [Errno 2] No such file or directory: '\\\\10.24.0.6\\tce$\\Informacoes_PDF\\GCADE\\GCADE_700896_2012_0017.pdf'


Multiple definitions in dictionary at byte 0xa59cf for key /Im118
Multiple definitions in dictionary at byte 0xa6587 for key /Im481
Multiple definitions in dictionary at byte 0xa65c3 for key /Im485
Multiple definitions in dictionary at byte 0xa6a46 for key /Im556
Multiple definitions in dictionary at byte 0xa6c44 for key /Im587
Multiple definitions in dictionary at byte 0xa8379 for key /Im984
Multiple definitions in dictionary at byte 0xa8863 for key /Im1059
Multiple definitions in dictionary at byte 0xa8984 for key /Im1076
Multiple definitions in dictionary at byte 0xa8a0c for key /Im1084
Multiple definitions in dictionary at byte 0xa9202 for key /Im475
Multiple definitions in dictionary at byte 0xa9279 for key /Im1204
Multiple definitions in dictionary at byte 0xaa694 for key /Im87
Multiple definitions in dictionary at byte 0xab594 for key /Im992
Multiple definitions in dictionary at byte 0xab77d for key /Im87
Multiple definitions in dictionary at byte 0xac382 for key /Im1951
Multipl

Error reading \\10.24.0.6\tce$\Informacoes_PDF\secss\secss_011548_2009_0076.pdf: [Errno 2] No such file or directory: '\\\\10.24.0.6\\tce$\\Informacoes_PDF\\secss\\secss_011548_2009_0076.pdf'


Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)
Ignoring wrong pointing object 32 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 48 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)


In [59]:
llm = AzureChatOpenAI(model_name="gpt-4o")
def is_julgamento_tc(texto):
    try:
        prompt = PromptTemplate.from_template("""
        Você é um agente que analisa e resume informações de processos do Tribunal de Contas do Estado do Rio Grande do Norte.

        O texto da informação é o seguinte:
        "{input}"
                                              
        Defina se a informação é um parecer prévio ou não.
        Responda apenas com "Sim" ou "Não". Não forneça explicações ou justificativas.                                           

        Sua resposta:
        """)

        chain = prompt | llm
        msg_obj = chain.invoke(texto)
        return msg_obj.content
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None

In [60]:
import re

def is_parecer_previo_regex(row):
    texto = row['texto']
    evento = row['evento']
    try:
        # Regex pattern to match "parecer prévio" or similar phrases
        pattern = r'\b(parecer prévio|parecer prévio do tribunal|parecer prévio do tce)\b'
        if re.search(pattern, texto, re.IGNORECASE):
            return evento
        else:
            return 'Não'
    except Exception as e:
        print(f"Error processing {texto}: {e}")
        return None

In [61]:
infos_sessoes = all_informacoes_processos[all_informacoes_processos.setor.str.lower().isin(['secss', 'secpc'])]

In [62]:
infos_sessoes['parecer_previo'] = infos_sessoes.apply(is_parecer_previo_regex, axis=1)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_18036\1802911371.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  infos_sessoes['parecer_previo'] = infos_sessoes.apply(is_parecer_previo_regex, axis=1)


In [63]:
infos_sessoes[infos_sessoes['parecer_previo'] != 'Não'].iloc[0]

arquivo                                                secss_005968_2013_0039.pdf
evento                                                                         51
processo                                                              005968/2013
setor                                                                       secss
numero_processo                                                            005968
ano_processo                                                                 2013
ordem                                                                          39
data_resumo                                            2019-10-14 08:57:38.020000
resumo                                                             ACÓRDÃO PADRÃO
sigilo                                                                          N
usuario                                                               06242394464
data_ultima_atualizacao                                2019-10-14 08:57:38.020000
informacao_efetu

In [ ]:
df_lista.drop(columns=['parecer_previo','julgamento_tc', 'dano'], inplace=True)

In [67]:
df_lista = df_lista.merge(infos_sessoes[['processo', 'parecer_previo']], on='processo', how='left')

JULGAMENTO_TC = julgamento direto pelo TCE, se não houver parecer, vai ter julgamento
DANO = ligado a pessoa da lista

In [68]:
sql_ressarcimentos = f''' SELECT CONCAT(numero_processo, '/', ano_processo) AS processo, deb.valororiginaldebito as dano
FROM processo.dbo.Exe_Debito deb INNER JOIN
processo.dbo.Processos pro ON deb.IdProcessoOrigem = pro.IdProcesso
WHERE CONCAT(numero_processo, '/', ano_processo) IN ({','.join(map(lambda x: f"'{x}'", df_lista['processo']))})
AND deb.CodigoTipoDebito = 1'''
df_ressarcimentos = pd.read_sql(sql_ressarcimentos, conn)

In [69]:
import locale
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

'pt_BR.UTF-8'

In [70]:
df_ressarcimentos['dano'] = df_ressarcimentos['dano'].apply(lambda x: locale.currency(x, grouping=True, symbol=False))

In [71]:
df_ressarcimentos = df_ressarcimentos.groupby('processo')['dano'].agg(', '.join).reset_index()

In [72]:
df_ressarcimentos['dano'] = df_ressarcimentos['dano'].apply(lambda x: f'Ressarcimentos: {x}')

In [73]:
tab_lista = df_lista.merge(df_ressarcimentos, on='processo', how='left')

In [76]:
tab_lista['parecer_previo'].fillna('Não', inplace=True)

In [78]:
tab_lista['dano'].fillna('Sem dano', inplace=True)

In [79]:
tab_lista.to_excel('db/lista_eleitoral.xlsx', index=False)

In [ ]:
for processo in df_lista['processo']:
    infos_processo = all_informacoes_processos[all_informacoes_processos['processo'] == processo]
    print(processo, infos_processo.shape[0])
    if infos_processo.shape[0] > 1:
        print(infos_processo[['setor', 'numero_processo', 'ano_processo', 'ordem', 'resumo', 'parecer_previo']])
        print("----")